##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum advantage in learning from experiments

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/quantum_advantage_in_learning_from_experiments"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/quantum/blob/master/docs/tutorials/quantum_advantage_in_learning_from_experiments.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/quantum/blob/master/docs/tutorials/quantum_advantage_in_learning_from_experiments.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/quantum/docs/tutorials/quantum_advantage_in_learning_from_experiments.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows the experiments of <a target="_blank" href="https://arxiv.org/abs/2112.00778" class="external">Quantum advantage in learning from experiments</a>.

## Setup

In [ ]:
!pip install tensorflow==2.4.1

Install TensorFlow Quantum:

In [ ]:
!pip install tensorflow-quantum

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

Now import TensorFlow and the module dependencies:

In [15]:
import tensorflow as tf
import cirq
from cirq_google import SycamoreGate
import numpy as np
import sympy
import functools
import os

## 1. The Basics

In [18]:
@functools.lru_cache(maxsize=128)
def _load_circuit(fname: str) -> cirq.Circuit:
    with open(fname, "r") as f:
        return cirq.read_json(f)

def un_bell_pair_block(qubits):
    """Un compute a bell pair on two qubits.

    Enacts CNOT(a, b) + H(a) using SycamoreGates and single qubit operations.

    Args:
        qubits: The qubits to un-prepare the bell pair on.

    Returns:
         A list of `cirq.Operations` realizing the operation.
    """
    mapped_circuit = _load_circuit("syc_un_bell_pair.json").transform_qubits(
        {cirq.GridQubit(0, 0): qubits[0], cirq.GridQubit(0, 1): qubits[1]}
    )
    return mapped_circuit.all_operations()

def inv_z_basis_gate(pauli):
    """Returns inverse Z basis transformation ops for a given Pauli.

    Args:
        pauli: Python str representing a single pauli.

    Returns:
        Corresponding `cirq.Gate` to do the inverse basis conversion.
    """
    if pauli == "I" or pauli == "Z":
        return cirq.I
    if pauli == "X":
        return cirq.H
    if pauli == "Y":
        # S^dag H to get to computational, H S to go back.
        return cirq.PhasedXZGate(
            axis_phase_exponent=-0.5, x_exponent=0.5, z_exponent=-0.5
        )
    raise ValueError("Invalid Pauli.")

def flatten_circuit(circuit: cirq.Circuit) -> cirq.Circuit:
    """Pack operations in circuit to the left as far as possible.

    Args:
        circuit: `cirq.Circuit` who's operations will be packed.

    Returns:
        A `cirq.Circuit` with operations packed to the left as
        far as possible.
    """
    return cirq.Circuit([op for mom in circuit for op in mom])

    
def build_circuit(
    qubit_pairs,
    pauli,
    n_shots,
    rand_state):
    """Create I + P problem circuit between qubit pairs.

    Args:
        qubit_pairs: List of qubit pairs.
        pauli: Python str containing characters 'I', 'X', 'Y' or 'Z'.
        n_shots: Number of repetitions to generate for sweeps.

    Returns:
        A (circuit, sweep) tuple, runnable using `run_sweep`.
    """
    a_qubits = [pair[0] for pair in qubit_pairs]
    b_qubits = [pair[1] for pair in qubit_pairs]
    all_qubits = np.concatenate(qubit_pairs)

    flip_params = sympy.symbols(f"param_0:{len(qubit_pairs) * 2}")

    # Add X flips.
    ret_circuit = cirq.Circuit(cirq.X(q) ** p for q, p in zip(all_qubits, flip_params))

    # Add basis turns a and b.
    ret_circuit += [
        inv_z_basis_gate(p)(q) for q, p in zip(a_qubits, pauli)
    ]
    ret_circuit += [
        inv_z_basis_gate(p)(q) for q, p in zip(b_qubits, pauli)
    ]

    # Add un-bell pair.
    ret_circuit += [un_bell_pair_block(pair) for pair in qubit_pairs]

    # Add measurements.
    for i, qubit in enumerate(all_qubits):
        ret_circuit += cirq.measure(qubit, key=f"q{i}")

    # Merge single qubit operations, flatten moments and align measurements.
    cirq.merge_single_qubit_gates_into_phxz(ret_circuit)
    cirq.DropEmptyMoments().optimize_circuit(circuit=ret_circuit)
    ret_circuit = flatten_circuit(ret_circuit)
    cirq.SynchronizeTerminalMeasurements().optimize_circuit(circuit=ret_circuit)

    # Create randomized flippings. These flippings will contain values of 1,0.
    # which will turn the X gates on or off.
    params = circuit_blocks.create_randomized_sweeps(
        pauli, flip_params, n_shots, rand_state
    )
    logging.debug(
        f"Generated circuit w/ depth {len(ret_circuit)} and {len(params)} sweeps."
    )
    return ret_circuit, params



In [19]:
rand_source = np.random.RandomState(1234)
n_paulis = 7
n = 3
n_shots = 100

paulis = np.array(["X", "Y", "Z", "I"])
pauli_strings = rand_source.choice(a=paulis, size=(n_paulis, n), replace=True)

system_pairs = [(cirq.GridQubit(0, i), cirq.GridQubit(1, i)) for i in range(n)]

for pauli in pauli_strings:
    print(pauli)
    
    circuit, sweeps = build_circuit(system_pairs, pauli, n_shots, rand_source)

['I' 'I' 'Z']


NameError: name 'circuit_blocks' is not defined